In [1]:
#libraryをimport
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#変数urlにSUUMOホームページをのURLを格納する（ユーザーが好みそうな新築・都心を条件設定）
url='https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=013&rn=0005&rn=0125&rn=0160&rn=0010&rn=0015&rn=0020&rn=0025&rn=0030&rn=0040&ek=012505180&ek=012504030&ek=012506530&ek=012510270&ek=012540650&ek=012520110&ek=012531160&ek=012523500&ek=012584570&ek=012517460&ek=012505480&ek=012508940&cb=0.0&ct=20.0&mb=0&mt=9999999&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=5&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=04&page={}'

#リストを作成、7864件のうち20ページ目までを繰り返し取得
property_list = []

for i in range(1,20):
    print(len(property_list))
    target_url = url.format(i)
    print(target_url)
    r = requests.get(target_url)

    #scraping負荷を減らす
    sleep(1)
    soup = BeautifulSoup(r.text)

    #全てのcasseteitem情報を取得
    contents = soup.find_all('div', class_='cassetteitem')

    #各物件情報をforループで取得する
    for content in contents:

        #物件情報を取得しておく
        detail = content.find('div', class_='cassetteitem-detail')

        #物件名と住所を取得
        title = detail.find('div', class_='cassetteitem_content-title').text
        station_near = detail.find('div', class_='cassetteitem_detail-text').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text

        #各部屋情報(階数、家賃、敷金＆礼金、広さ)を取得
        table = content.find('table', class_='cassetteitem_other')
        tr_tags = table.find_all('tr', class_='js-cassette_link')
        floor, price, first_fee, capacity = tr_tags[0].find_all('td')[2:6]

        fee, management_fee = price.find_all('li')
        deposit, gratuity = first_fee.find_all('li')
        madori, menseki = capacity.find_all('li')

        property = {
            '物件名': title,
            '最寄り駅': station_near,
            '住所': address,
            '間取り': madori.text,
            '家賃': fee.text,
            '階': floor.text
        }

        #取得した辞書をproperty_listに格納する
        property_list.append(property)

0
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=013&rn=0005&rn=0125&rn=0160&rn=0010&rn=0015&rn=0020&rn=0025&rn=0030&rn=0040&ek=012505180&ek=012504030&ek=012506530&ek=012510270&ek=012540650&ek=012520110&ek=012531160&ek=012523500&ek=012584570&ek=012517460&ek=012505480&ek=012508940&cb=0.0&ct=20.0&mb=0&mt=9999999&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=5&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=04&page=1
30
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=013&rn=0005&rn=0125&rn=0160&rn=0010&rn=0015&rn=0020&rn=0025&rn=0030&rn=0040&ek=012505180&ek=012504030&ek=012506530&ek=012510270&ek=012540650&ek=012520110&ek=012531160&ek=012523500&ek=012584570&ek=012517460&ek=012505480&ek=012508940&cb=0.0&ct=20.0&mb=0&mt=9999999&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=5&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=04&page=2
60
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=013&rn=

In [18]:
df = pd.DataFrame(property_list)
df.head()

,物件名,最寄り駅,住所,間取り,家賃,階
0,ｔｏｇｇｌｅ文京大塚,東京メトロ丸ノ内線/新大塚駅 歩5分,東京都文京区大塚５,2DK,18.8万円,\r\n\t\t\t\t\t\t\t\t\t\t\t1階
1,東京メトロ丸ノ内線 新大塚駅 4階建 新築,東京メトロ丸ノ内線/新大塚駅 歩5分,東京都文京区大塚５,2DK,18.8万円,\r\n\t\t\t\t\t\t\t\t\t\t\t1階
2,ディアー ファースト,東京メトロ東西線/葛西駅 歩4分,東京都江戸川区東葛西５,2LDK,15.4万円,\r\n\t\t\t\t\t\t\t\t\t\t\t2階
3,東京メトロ丸ノ内線 方南町駅 3階建 新築,東京メトロ丸ノ内線/方南町駅 歩4分,東京都中野区南台５,2LDK,20万円,\r\n\t\t\t\t\t\t\t\t\t\t\t3階
4,東京メトロ日比谷線 三ノ輪駅 7階建 新築,東京メトロ日比谷線/三ノ輪駅 歩5分,東京都荒川区東日暮里１,2LDK,17.9万円,\r\n\t\t\t\t\t\t\t\t\t\t\t2階


In [19]:
#階列から、「階」を削除
df['階'] = df['階'].apply(lambda x: x.replace('階', ''))
#家賃から、「万円」を削除
df['家賃'] = df['家賃'].apply(lambda x: x.replace('万円', ''))
#最寄駅から、駅までの距離だけを抽出し新規列に追加→最寄り駅の後ろに並び替える
df[['最寄り駅', '駅までの距離']] = df['最寄り駅'].str.split(' ', expand=True)
#最寄り駅から、「東京メトロ」を削除
df['最寄り駅'] = df['最寄り駅'].apply(lambda x: x.replace('東京メトロ', ''))
#住所から、「東京都」を削除
df['住所'] = df['住所'].apply(lambda x: x.replace('東京都', ''))
df = df.rename(columns={'駅までの距離': '駅までの距離（徒歩/分）'})

#順番を変更
new_column_order = ['物件名', '最寄り駅', '駅までの距離（徒歩/分）', '間取り','階','家賃', '住所']
#新しいdfを作成
df = df[new_column_order]
df.head()

,物件名,最寄り駅,駅までの距離（徒歩/分）,間取り,階,家賃,住所
0,ｔｏｇｇｌｅ文京大塚,丸ノ内線/新大塚駅,歩5分,2DK,\r\n\t\t\t\t\t\t\t\t\t\t\t1,18.8,文京区大塚５
1,東京メトロ丸ノ内線 新大塚駅 4階建 新築,丸ノ内線/新大塚駅,歩5分,2DK,\r\n\t\t\t\t\t\t\t\t\t\t\t1,18.8,文京区大塚５
2,ディアー ファースト,東西線/葛西駅,歩4分,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t2,15.4,江戸川区東葛西５
3,東京メトロ丸ノ内線 方南町駅 3階建 新築,丸ノ内線/方南町駅,歩4分,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t3,20,中野区南台５
4,東京メトロ日比谷線 三ノ輪駅 7階建 新築,日比谷線/三ノ輪駅,歩5分,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t2,17.9,荒川区東日暮里１


In [20]:
#駅までの距離から、歩と分を消す
df['駅までの距離（徒歩/分）'] = df['駅までの距離（徒歩/分）'].apply(lambda x: x.replace('歩', ''))
df['駅までの距離（徒歩/分）'] = df['駅までの距離（徒歩/分）'].apply(lambda x: x.replace('分', ''))
df.head()

,物件名,最寄り駅,駅までの距離（徒歩/分）,間取り,階,家賃,住所
0,ｔｏｇｇｌｅ文京大塚,丸ノ内線/新大塚駅,5,2DK,\r\n\t\t\t\t\t\t\t\t\t\t\t1,18.8,文京区大塚５
1,東京メトロ丸ノ内線 新大塚駅 4階建 新築,丸ノ内線/新大塚駅,5,2DK,\r\n\t\t\t\t\t\t\t\t\t\t\t1,18.8,文京区大塚５
2,ディアー ファースト,東西線/葛西駅,4,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t2,15.4,江戸川区東葛西５
3,東京メトロ丸ノ内線 方南町駅 3階建 新築,丸ノ内線/方南町駅,4,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t3,20,中野区南台５
4,東京メトロ日比谷線 三ノ輪駅 7階建 新築,日比谷線/三ノ輪駅,5,2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t2,17.9,荒川区東日暮里１


In [21]:
# 物件名と最寄り駅の共通部分を特定する
def find_common_part(title, station_near):
    common_parts = []
    for word in title.split():
        if word in station_near:
            common_parts.append(word)
    return " ".join(common_parts)

# 共通部分をdfとして格納
df['共通部分'] = df.apply(lambda x: find_common_part(x['物件名'], x['最寄り駅']), axis=1)

# 物件名から共通部分削除したものをdfに格納
df['物件名（共通部分削除後）'] = df.apply(lambda x: x['物件名'].replace(x['共通部分'], ''), axis=1)

# 共通部分削除後の物件名を、元の物件名に置き換える
df['物件名'] = df['物件名（共通部分削除後）']

# 共通部分削除のカラムを削除する
df = df.drop(columns=['共通部分', '物件名（共通部分削除後）'])

# 最寄り駅名カラムを、路線名と駅名で分割
df[['路線名', '駅名']] = df['最寄り駅'].str.split('/', expand=True)

# 最寄り駅カラムと路線名、駅名カラムを入れ替え、最寄り駅カラムを削除
df = df.drop('最寄り駅', axis=1)
column_order = ['物件名', '路線名', '駅名', '駅までの距離（徒歩/分）', '間取り', '階', '家賃', '住所']
df = df[column_order]

# 不要なスペースをすべて削除
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.head()

,物件名,路線名,駅名,駅までの距離（徒歩/分）,間取り,階,家賃,住所
0,ｔｏｇｇｌｅ文京大塚,丸ノ内線,新大塚駅,5,2DK,1,18.8,文京区大塚５
1,東京メトロ丸ノ内線 4階建 新築,丸ノ内線,新大塚駅,5,2DK,1,18.8,文京区大塚５
2,ディアー ファースト,東西線,葛西駅,4,2LDK,2,15.4,江戸川区東葛西５
3,東京メトロ丸ノ内線 3階建 新築,丸ノ内線,方南町駅,4,2LDK,3,20,中野区南台５
4,東京メトロ日比谷線 7階建 新築,日比谷線,三ノ輪駅,5,2LDK,2,17.9,荒川区東日暮里１


In [22]:
# 重複した行を特定する条件を指定
duplicate_condition = df.duplicated(subset=['住所', '家賃', '階'], keep=False)

# 重複した行を抽出
duplicates = df[duplicate_condition]

# 重複した行を表示
print(duplicates)

                     物件名   路線名     駅名 駅までの距離（徒歩/分）   間取り  階    家賃        住所
0             ｔｏｇｇｌｅ文京大塚  丸ノ内線   新大塚駅            5   2DK  1  18.8    文京区大塚５
1      東京メトロ丸ノ内線  4階建 新築  丸ノ内線   新大塚駅            5   2DK  1  18.8    文京区大塚５
11            クレイシアIDZ浅草   銀座線   田原町駅            5    2K  2  10.3   台東区元浅草４
12     東京メトロ銀座線  11階建 新築   銀座線   田原町駅            5    2K  2  10.3   台東区元浅草４
13              リックアート東陽   東西線    木場駅            5   2DK  2  15.1    江東区東陽３
..                   ...   ...    ...          ...   ... ..   ...       ...
563  東京メトロ東西線  10階建 築32年   東西線    葛西駅            2  2LDK  5  10.3  江戸川区東葛西６
564                ハイツ疋田  丸ノ内線  新高円寺駅            5   2DK  1   9.9   杉並区松ノ木３
565  東京メトロ丸ノ内線  3階建 築32年  丸ノ内線  新高円寺駅            5   2DK  1   9.9   杉並区松ノ木３
566               イーストビル  千代田線    綾瀬駅            4   2DK  3   8.6    足立区綾瀬２
567  東京メトロ千代田線  7階建 築32年  千代田線    綾瀬駅            4    2K  3   8.6    足立区綾瀬２

[302 rows x 8 columns]


In [23]:
# 重複した行を削除
df = df.drop_duplicates(subset=['住所', '家賃', '階'], keep='first')
df.head()

,物件名,路線名,駅名,駅までの距離（徒歩/分）,間取り,階,家賃,住所
0,ｔｏｇｇｌｅ文京大塚,丸ノ内線,新大塚駅,5,2DK,1,18.8,文京区大塚５
2,ディアー ファースト,東西線,葛西駅,4,2LDK,2,15.4,江戸川区東葛西５
3,東京メトロ丸ノ内線 3階建 新築,丸ノ内線,方南町駅,4,2LDK,3,20,中野区南台５
4,東京メトロ日比谷線 7階建 新築,日比谷線,三ノ輪駅,5,2LDK,2,17.9,荒川区東日暮里１
5,PETITO南台,丸ノ内線,方南町駅,4,2LDK,2,18,中野区南台５


In [24]:
# dfをcsvに出力
df.to_csv('suumo_313try.csv', index=None, encoding='utf-8-sig')

ここからGCP連携→スプシ格納（pip install実施済み）

In [31]:
#ライブラリをインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

In [30]:
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('--------------------------', scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open('STEP3 1-3 SUUMO')

In [ ]:
# DataFrameのデータをリストに変換（カラム名を含む）
df_data_with_columns = [df.columns.tolist()] + df.values.tolist()

# データをスプレッドシートに挿入
worksheet = spreadsheet.get_worksheet(0)  # ワークシートのインデックスを指定
worksheet.insert_rows(df_data_with_columns, 2)  # データを挿入
# 空白行を削除
worksheet.delete_row(1)  # 1行目を削除
#提出用に新しいシートに格納
spreadsheet.add_worksheet(title="STEP3 1-3 submit", rows=50, cols=10)
set_with_dataframe(spreadsheet.worksheet("STEP3 1-3 submit"), df, include_index=True)